In [1]:
import pandas as pd
import requests
import json
import os
import io
from datetime import datetime
import numpy as np

API_KEY = os.getenv("API_KEY")
url = 'https://data-service.beatthebookie.blog/data'
headers = {"x-api-key": API_KEY}

# Function to fetch data for a specific division and season
def fetch_data(division, season):
    params = {
        'division': division,
        'season': season
    }
    response = requests.get(url, headers=headers, params=params)
    if response.status_code == 200:
        return pd.read_json(io.StringIO(response.content.decode('utf-8')))
    else:
        print(f"Error fetching {division} {season}: {response.status_code}")
        print(response.content.decode('utf-8'))
        return pd.DataFrame()

# Fetch data for all combinations
seasons = ['2023_2024', '2024_2025']
divisions = ['Premier League', 'Championship']
dataframes = []

for division in divisions:
    for season in seasons:
        df = fetch_data(division, season)
        if not df.empty:
            dataframes.append(df)

# Combine all dataframes
if dataframes:
    df = pd.concat(dataframes, ignore_index=True)
    
    # Convert match_date to datetime
    df['match_date'] = pd.to_datetime(df['match_date'])
    
    # Filter for dates after January 11, 2024
    df = df[df["match_date"] > '2024-01-11']
    
    # Get unique teams from 2024/25 Premier League
    prem_teams_25 = df[
        (df['season'] == '2024_2025') & 
        (df['division'] == 'Premier League')
    ]
    prem_teams_25 = pd.concat([
        prem_teams_25['home_team'], 
        prem_teams_25['away_team']
    ]).unique()
    
    # Display the results
    print("\nSample of filtered matches:")
    print(df[["season", "division", "match_date", "home_team", "away_team", 
              "home_goals", "home_xgoals", "away_goals", "away_xgoals"]].tail())
    
    print("\nTotal matches:", len(df))
    print("\nUnique seasons:", df['season'].unique())
    print("Unique divisions:", df['division'].unique())
else:
    print("No data was successfully fetched")


Sample of filtered matches:
        season      division match_date        home_team      away_team  \
1447  20242025  Championship 2025-01-05       Sunderland  FC Portsmouth   
1448  20242025  Championship 2025-01-06              QPR          Luton   
1449  20242025  Championship 2025-01-14          Cardiff        Watford   
1450  20242025  Championship 2025-01-14  Plymouth Argyle  Oxford United   
1451  20242025  Championship 2025-01-15        Blackburn  FC Portsmouth   

      home_goals  home_xgoals  away_goals  away_xgoals  
1447           1          2.3           0          0.5  
1448           2          2.2           1          2.1  
1449           1          0.7           1          1.0  
1450           1          0.6           1          0.4  
1451           3          1.3           0          1.0  

Total matches: 942

Unique seasons: [20232024 20242025]
Unique divisions: ['Premier League' 'Championship']


In [2]:
# Apply averaged penalty when Championship match contains two promoted teams.

# Penalty xG = 0.665
# Penalty xGA = 1.465

# mask_both_prem = (df['division'] == 'Championship') & (df['home_team'].isin(prem_teams_25)) & (df['away_team'].isin(prem_teams_25))
# # #df.loc[mask_both_prem, 'home_goals'] *= 0.661
# # #df.loc[mask_both_prem, 'away_goals'] *= 0.661
# df.loc[mask_both_prem, 'home_xgoals'] *= 0.665
# df.loc[mask_both_prem, 'away_xgoals'] *= 0.655

# # Adjust performance penalty to championship games when one team is a promoted team.
# mask_home = (df['division'] == 'Championship') & (df['home_team'].isin(prem_teams_25)) & ~(df['away_team'].isin(prem_teams_25))
# # #df.loc[mask_home, 'home_goals'] *= 0.661
# # #df.loc[mask_home, 'away_goals'] *= 2.060
# df.loc[mask_home, 'home_xgoals'] *= 0.665
# df.loc[mask_home, 'away_xgoals'] *= 1.465

# mask_away = (df['division'] == 'Championship') & (df['away_team'].isin(prem_teams_25)) & ~(df['home_team'].isin(prem_teams_25))
# # #df.loc[mask_away, 'home_goals'] *= 2.060
# # #df.loc[mask_away, 'away_goals'] *= 0.661
# df.loc[mask_away, 'home_xgoals'] *= 1.465
# df.loc[mask_away, 'away_xgoals'] *= 0.665


# df[['home_goals', 'away_goals']] = df[['home_goals', 'away_goals']].round()


In [3]:
def decay(xi, t):
    return np.exp(-xi * t)

def rho_correction(goals_home, goals_away, home_exp, away_exp, rho):
    if goals_home == 0 and goals_away == 0:
        return 1 - (home_exp * away_exp * rho)
    elif goals_home == 0 and goals_away == 1:
        return 1 + (home_exp * rho)
    elif goals_home == 1 and goals_away == 0:
        return 1 + (away_exp * rho)
    elif goals_home == 1 and goals_away == 1:
        return 1 - rho
    else:
        return 1.0

In [4]:
from scipy.optimize import minimize
from scipy.stats import norm, poisson

def log_likelihood_xg(
    xG_home_observed,
    xG_away_observed,
    home_attack,
    home_defence,
    away_attack,
    away_defence,
    home_advantage,
    weight,
    sigma=1.0
):
    xG_expectation_home = np.exp(home_attack + away_defence + home_advantage)
    xG_expectation_away = np.exp(away_attack + home_defence)

    home_llk = norm.pdf(xG_home_observed, loc=xG_expectation_home, scale=sigma)
    away_llk = norm.pdf(xG_away_observed, loc=xG_expectation_away, scale=sigma)

    if xG_expectation_home < 0 or xG_expectation_away < 0:
        return 10000

    log_llk = weight * (np.log(home_llk) + np.log(away_llk))

    return -log_llk


def fit_xG_model(df, xi=0.0001, sigma=1.0):
    teams = np.sort(np.unique(np.concatenate([df["home_team"], df["away_team"]])))
    n_teams = len(teams)
    
    df["league_strength"] = df["division"].apply(lambda x: 1 if x == "Premier League" else 0.615) 
    df["days_since"] = (df["match_date"].max() - df["match_date"]).dt.days
    df["weight"] = decay(xi, df["days_since"]) * df["league_strength"]

    params = np.concatenate(
        (
            np.random.uniform(0.5, 1.5, (n_teams)),  # attack strength
            np.random.uniform(0, -1, (n_teams)),  # defence strength
            [0.25],  # home advantage
        )
    )

    bounds = [(None, None)] * (2 * n_teams) + [(None, None)]

    def _fit(params, df, teams, sigma):
        attack_params = dict(zip(teams, params[:n_teams]))
        defence_params = dict(zip(teams, params[n_teams : (2 * n_teams)]))
        home_advantage = params[-1]

        llk = list()
        for idx, row in df.iterrows():
            tmp = log_likelihood_xg(
                row["home_xgoals"],
                row["away_xgoals"],
                attack_params[row["home_team"]],
                defence_params[row["home_team"]],
                attack_params[row["away_team"]],
                defence_params[row["away_team"]],
                home_advantage,
                row["weight"],
                sigma=sigma
            )
            llk.append(tmp)

        return np.sum(llk)

    options = {
        "maxiter": 100,
        "disp": False,
    }

    constraints = [{"type": "eq", "fun": lambda x: sum(x[:n_teams]) - n_teams}]

    res = minimize(
        _fit,
        params,
        args=(df, teams, sigma),
        constraints=constraints,
        bounds=bounds,
        method='L-BFGS-B',
        options=options,
    )

    model_params = dict(
        zip(
            ["attack_" + team for team in teams]
            + ["defence_" + team for team in teams]
            + ["home_adv"],
            res["x"],
        )
    )

    # Prepare table data
    attack_values = [model_params[f'attack_{team}'] for team in teams]
    defence_values = [model_params[f'defence_{team}'] for team in teams]
    home_adv = model_params['home_adv']

    # Create DataFrame for attack and defense
    team_strength_df_xg = pd.DataFrame({
        'Team': teams,
        'Attack Strength': attack_values,
        'Defense Strength': defence_values
    })

    # Add home advantage to the DataFrame
    additional_params_df = pd.DataFrame({
        'Parameter': ['Home Advantage'],
        'Value': [home_adv]
    })

    # Print DataFrames
    print("Team Strength (Attack and Defense):")
    print(team_strength_df_xg)
    print("\nAdditional Parameters (Home Advantage):")
    print(additional_params_df)

    print("Log Likelihood: ", res["fun"])

    team_strength_df_xg.to_csv("predictions/xg_team_strength_jan25.csv")

    return model_params


xg_model_params = fit_xG_model(df, xi=0.001, sigma=1.0)

C:\Users\Owner\AppData\Local\Temp\ipykernel_14860\560937854.py:76: RuntimeWarning: Method L-BFGS-B cannot handle constraints.
  res = minimize(


Team Strength (Attack and Defense):
                Team  Attack Strength  Defense Strength
0            Arsenal         1.501859         -1.424345
1        Aston Villa         1.244430         -0.790665
2         Birmingham         0.374819         -0.552072
3          Blackburn         0.505048         -0.553800
4        Bournemouth         1.332442         -0.899661
5          Brentford         1.284307         -0.747773
6           Brighton         1.076754         -0.874181
7       Bristol City         0.597954         -0.520383
8            Burnley         0.618038         -0.652147
9            Cardiff         0.341861         -0.268152
10           Chelsea         1.520957         -0.832178
11          Coventry         0.743367         -0.568305
12    Crystal Palace         1.173917         -0.854595
13             Derby         0.440621         -0.535279
14           Everton         0.932864         -0.800186
15     FC Portsmouth         0.438297         -0.185111
16          

In [5]:
def log_likelihood_goals(
    goals_home_observed,
    goals_away_observed,
    home_attack,
    home_defence,
    away_attack,
    away_defence,
    home_advantage,
    rho,
    weight
):
    goal_expectation_home = np.exp(home_attack + away_defence + home_advantage)
    goal_expectation_away = np.exp(away_attack + home_defence)

    home_llk = poisson.pmf(goals_home_observed, goal_expectation_home)
    away_llk = poisson.pmf(goals_away_observed, goal_expectation_away)
    adj_llk = rho_correction(
        goals_home_observed,
        goals_away_observed,
        goal_expectation_home,
        goal_expectation_away,
        rho,
    )

    if goal_expectation_home < 0 or goal_expectation_away < 0 or adj_llk < 0:
        return 10000

    log_llk = weight * (np.log(home_llk) + np.log(away_llk) + np.log(adj_llk))

    return -log_llk

def fit_poisson_model(df, xi=0.0001):
    teams = np.sort(np.unique(np.concatenate([df["home_team"], df["away_team"]])))
    n_teams = len(teams)
    
    df["league_strength"] = df["division"].apply(lambda x: 1 if x == "Premier League" else 0.615) 
    df["days_since"] = (df["match_date"].max() - df["match_date"]).dt.days
    df["weight"] = decay(xi, df["days_since"]) * df["league_strength"]

    params = np.concatenate(
        (
            np.random.uniform(0.5, 1.5, (n_teams)),  # attack strength
            np.random.uniform(0, -1, (n_teams)),  # defence strength
            [0.25],  # home advantage
            [-0.1], # rho
        )
    )

    bounds = [(None, None)] * (2 * n_teams) + [(None, None), (-1, 1)]

    def _fit(params, df, teams):
        attack_params = dict(zip(teams, params[:n_teams]))
        defence_params = dict(zip(teams, params[n_teams : (2 * n_teams)]))
        home_advantage = params[-2]
        rho = params[-1]

        llk = list()
        for idx, row in df.iterrows():
            tmp = log_likelihood_goals(
                row["home_goals"],
                row["away_goals"],
                attack_params[row["home_team"]],
                defence_params[row["home_team"]],
                attack_params[row["away_team"]],
                defence_params[row["away_team"]],
                home_advantage,
                rho,
                row["weight"]
            )
            llk.append(tmp)

        return np.sum(llk)

    options = {
        "maxiter": 100,
        "disp": False,
    }

    constraints = [{"type": "eq", "fun": lambda x: sum(x[:n_teams]) - n_teams}]

    res = minimize(
        _fit,
        params,
        args=(df, teams),
        constraints=constraints,
        method='L-BFGS-B',
        options=options,
        bounds=bounds
    )

    model_params = dict(
        zip(
            ["attack_" + team for team in teams]
            + ["defence_" + team for team in teams]
            + ["home_adv", "rho"],
            res["x"],
        )
    )

    # Prepare table data
    attack_values = [model_params[f'attack_{team}'] for team in teams]
    defence_values = [model_params[f'defence_{team}'] for team in teams]
    home_adv = model_params['home_adv']
    rho = model_params['rho']

    # Create DataFrame for attack and defense
    team_strength_df_gls = pd.DataFrame({
        'Team': teams,
        'Attack Strength': attack_values,
        'Defense Strength': defence_values
    })

    # Add home advantage and rho to the DataFrame
    additional_params_df = pd.DataFrame({
        'Parameter': ['Home Advantage', 'Rho'],
        'Value': [home_adv, rho]
    })

    # Print DataFrames
    print("Team Strength (Attack and Defense):")
    print(team_strength_df_gls)
    print("\nAdditional Parameters (Home Advantage and Rho):")
    print(additional_params_df)

    print("Log Likelihood: ", res["fun"])

    team_strength_df_gls.to_csv("predictions/gls_team_strength_jan25.csv")

    return model_params

goals_model_params = fit_poisson_model(df, xi=0.001)

C:\Users\Owner\AppData\Local\Temp\ipykernel_14860\3060770712.py:81: RuntimeWarning: Method L-BFGS-B cannot handle constraints.
  res = minimize(


Team Strength (Attack and Defense):
                Team  Attack Strength  Defense Strength
0            Arsenal         1.632109         -1.632893
1        Aston Villa         1.275103         -0.810314
2         Birmingham         0.289568         -0.500919
3          Blackburn         0.464270         -0.662022
4        Bournemouth         1.157648         -1.011146
5          Brentford         1.379156         -0.764583
6           Brighton         1.071689         -0.971751
7       Bristol City         0.495764         -0.541429
8            Burnley         0.686874         -0.917641
9            Cardiff         0.328787         -0.157808
10           Chelsea         1.579311         -0.955104
11          Coventry         0.673423         -0.267661
12    Crystal Palace         1.156138         -0.990574
13             Derby         0.508064         -0.400718
14           Everton         0.581002         -1.109720
15     FC Portsmouth         0.555756         -0.100847
16          

In [15]:
def predict(params, home_team, away_team):
    home_attack = params["attack_" + home_team]
    home_defence = params["defence_" + home_team]
    away_attack = params["attack_" + away_team]
    away_defence = params["defence_" + away_team]
    home_advantage = params["home_adv"]
    rho = params.get("rho", 0)

    home_goal_expectation = np.exp(home_attack + away_defence + home_advantage)
    away_goal_expectation = np.exp(away_attack + home_defence)

    home_probs = poisson.pmf(range(10), home_goal_expectation)
    away_probs = poisson.pmf(range(10), away_goal_expectation)

    m = np.outer(home_probs, away_probs)

    m[0, 0] *= 1 - home_goal_expectation * away_goal_expectation * rho
    m[0, 1] *= 1 + home_goal_expectation * rho
    m[1, 0] *= 1 + away_goal_expectation * rho
    m[1, 1] *= 1 - rho    

    home = np.sum(np.tril(m, -1)) 
    draw = np.sum(np.diag(m)) 
    away = np.sum(np.triu(m, 1))

    total_prob = home + draw + away
    home /= total_prob
    draw /= total_prob
    away /= total_prob 

    # Calculate the probability of a clean sheet for the home team (away team scores 0)
    home_clean_sheet_prob = m[:, 0].sum() 

    # Calculate the probability of a clean sheet for the away team (home team scores 0)
    away_clean_sheet_prob = m[0, :].sum() 

    # Calculate the probability of the home team scoring 3 or more goals
    home_3_plus_goals_prob = home_probs[3:].sum() 

    # Calculate the probability of the away team scoring 3 or more goals
    away_3_plus_goals_prob = away_probs[3:].sum() 


    return {
        "home_win_prob": home.round(2),
        "draw_prob": draw.round(2),
        "away_win_prob": away.round(2),
        "home_clean_sheet_prob": home_clean_sheet_prob.round(2),
        "away_clean_sheet_prob": away_clean_sheet_prob.round(2),
        "home_goal_expectation": home_goal_expectation.round(2),
        "away_goal_expectation": away_goal_expectation.round(2),
        "home_3_plus_goals_prob": home_3_plus_goals_prob.round(2),
        "away_3_plus_goals_prob": away_3_plus_goals_prob.round(2)
    }

In [7]:
# Initialize lists to store predictions
xg_home_preds = []
xg_away_preds = []
goals_home_preds = []
goals_away_preds = []

# Iterate through the games in your dataset
for idx, row in df.iterrows():
    xg_pred = predict(xg_model_params, row['home_team'], row['away_team'])
    goals_pred = predict(goals_model_params, row['home_team'], row['away_team'])

    xg_home_preds.append(xg_pred['home_goal_expectation'])
    xg_away_preds.append(xg_pred['away_goal_expectation'])
    goals_home_preds.append(goals_pred['home_goal_expectation'])
    goals_away_preds.append(goals_pred['away_goal_expectation'])

# Add these predictions back to the DataFrame
df['xg_home_pred'] = xg_home_preds
df['xg_away_pred'] = xg_away_preds
df['goals_home_pred'] = goals_home_preds
df['goals_away_pred'] = goals_away_preds

# Calculate absolute errors for xG predictions
df['xg_home_error'] = abs(df['home_xgoals'] - df['xg_home_pred'])
df['xg_away_error'] = abs(df['away_xgoals'] - df['xg_away_pred'])

# Calculate absolute errors for goals predictions
df['goals_home_error'] = abs(df['home_goals'] - df['goals_home_pred'])
df['goals_away_error'] = abs(df['away_goals'] - df['goals_away_pred'])


# Calculate MAE for xG predictions
xg_mae_home = df['xg_home_error'].mean()
xg_mae_away = df['xg_away_error'].mean()

# Calculate MAE for goals predictions
goals_mae_home = df['goals_home_error'].mean()
goals_mae_away = df['goals_away_error'].mean()

# Combine MAEs for home and away
xg_mae_total = (xg_mae_home + xg_mae_away) / 2
goals_mae_total = (goals_mae_home + goals_mae_away) / 2

print(f"xG MAE (home): {xg_mae_home}")
print(f"xG MAE (away): {xg_mae_away}")
print(f"xG MAE (total): {xg_mae_total}")

print(f"Goals MAE (home): {goals_mae_home}")
print(f"Goals MAE (away): {goals_mae_away}")
print(f"Goals MAE (total): {goals_mae_total}")

xG MAE (home): 0.5810605164543524
xG MAE (away): 0.5392058908704883
xG MAE (total): 0.5601332036624204
Goals MAE (home): 0.916900212314225
Goals MAE (away): 0.8265074309978769
Goals MAE (total): 0.871703821656051


In [18]:
import pandas as pd
import sys
import os


# Manually define fixtures for GW22
fixtures = [
    ("Newcastle", "Bournemouth"),
    ("Brentford", "Liverpool"),
    ("Leicester", "Fulham"),
    ("West Ham", "Crystal Palace"),
    ("Arsenal", "Aston Villa"),
    ("Everton", "Tottenham"),
    ("Man United", "Brighton"),
    ("Nott'm Forest", "Southampton"),
    ("Ipswich", "Man City"),
    ("Chelsea", "Wolves")
]

# Get predictions for each fixture
results = []

for fixture in fixtures:
    home_team, away_team = fixture
    
    xg_preds = predict(xg_model_params, home_team, away_team)
    goals_preds = predict(goals_model_params, home_team, away_team)
    
    weighted_preds = {}
    
    for key in xg_preds.keys():
        weighted_value = (0.7 * xg_preds[key]) + (0.3 * goals_preds[key])
        weighted_preds[key] = weighted_value
    
    results.append({
        'home_team': home_team,
        'away_team': away_team,
        **weighted_preds
    })

results_df = pd.DataFrame(results)


# Display the results DataFrame
display(results_df)


,home_team,away_team,home_win_prob,draw_prob,away_win_prob,home_clean_sheet_prob,away_clean_sheet_prob,home_goal_expectation,away_goal_expectation,home_3_plus_goals_prob,away_3_plus_goals_prob
0,Newcastle,Bournemouth,0.511,0.226,0.266,0.257,0.140,1.966,1.371,0.313,0.159
1,Brentford,Liverpool,0.196,0.186,0.615,0.083,0.261,1.339,2.466,0.149,0.444
2,Leicester,Fulham,0.248,0.236,0.516,0.167,0.306,1.192,1.808,0.117,0.273
3,West Ham,Crystal Palace,0.335,0.229,0.436,0.163,0.215,1.561,1.816,0.205,0.274
4,Arsenal,Aston Villa,0.748,0.154,0.095,0.451,0.084,2.541,0.798,0.468,0.044
5,Everton,Tottenham,0.286,0.251,0.466,0.184,0.281,1.295,1.700,0.144,0.247
6,Man United,Brighton,0.440,0.245,0.315,0.254,0.192,1.656,1.363,0.235,0.159
7,Nott'm Forest,Southampton,0.733,0.166,0.101,0.443,0.086,2.472,0.818,0.449,0.051
8,Ipswich,Man City,0.131,0.176,0.690,0.086,0.367,1.004,2.450,0.078,0.445
9,Chelsea,Wolves,0.753,0.140,0.100,0.348,0.054,2.944,1.052,0.561,0.089


In [19]:
from viz.model_pred_viz.gw_dict import gw_dict
current_date = pd.Timestamp.today()
for gw, date in gw_dict.items():
        if current_date >= pd.to_datetime(date):
            current_gw = gw
        else:
            break
print (current_gw)

results_df.to_csv(f"predictions/ensmeble_{current_gw}_preds.csv")

GW22
